In [1]:
import json
import pandas as pd
from collections import defaultdict

In [2]:
with open('976545.json') as f:
    j = json.load(f)

In [3]:
time_diffs = defaultdict(lambda: defaultdict(str))

for id, member in j['members'].items():
    name = member['name'] if member['name'] else id
    for day, times in member['completion_day_level'].items():
        if '1' in times and '2' in times:
            time_diffs[name][int(day)] = int(times['2']['get_star_ts']) - int(times['1']['get_star_ts'])

In [4]:
df = pd.DataFrame.from_dict(time_diffs, orient='index')
df.sort_index(key=lambda x: x.str.lower(), inplace=True)
df.sort_index(axis=1, inplace=True)

In [5]:
df

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
1002756,529.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1192974,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
982075,1281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999754,295.0,863.0,565.0,4370.0,585.0,3335.0,11017.0,3713.0,1238.0,NaN,...,NaN,2049.0,2943.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attoPascal,91.0,340.0,444.0,3061.0,1159.0,1469.0,426.0,3828.0,1344.0,9062.0,...,5411.0,113.0,979.0,2547.0,12209.0,653.0,5249.0,12934.0,647.0,54.0
baduncle93,131.0,1198.0,4705.0,3335.0,421.0,3716.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bernhard Jordan,191.0,373.0,2957.0,2467.0,767.0,917.0,451.0,5748.0,1237.0,18372.0,...,27656.0,527.0,830.0,60.0,1292.0,440.0,8127.0,1038.0,5826.0,58.0
Christoph D.,179.0,268.0,1150.0,3991.0,989.0,657.0,1464.0,2181.0,951.0,27676.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Der_Himbeerkuchen,1290.0,1152.0,340.0,3112.0,953.0,4407.0,3813.0,7782.0,2800.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
devgertschi,4325.0,758.0,2182.0,3132.0,571.0,1447.0,5655.0,2001.0,917.0,9714.0,...,4024.0,850.0,7207.0,4706.0,NaN,303.0,5141.0,NaN,NaN,NaN


In [6]:
num_members, num_days = df.shape
points = df[1].copy()
points[:] = 0

for day in range(1, num_days+1):
    max_points = num_members
    for name, time in df[day].sort_values().iteritems():
        if not pd.isna(time):
            points[name] += max_points
            max_points -= 1

In [7]:
points.sort_values(ascending=False)

michalrzak            610.0
attoPascal            605.0
Bernhard Jordan       597.0
devgertschi           473.0
felpower              417.0
Rosi-Eliz             399.0
Katakompe             384.0
Hablam                351.0
memoriesadrift        303.0
999754                268.0
Triarier              244.0
Christoph D.          233.0
thoosequa             222.0
trimaru               209.0
Simon Eckerstorfer    201.0
Der_Himbeerkuchen     141.0
Nico Schlauf          123.0
rpsdm                 122.0
baduncle93             99.0
zoidberg77             95.0
marc1729               83.0
noxeee                 82.0
PKlempe                82.0
rmezameza              34.0
1192974                29.0
1002756                14.0
megamopsy              11.0
982075                 10.0
Grarak                  7.0
Name: 1, dtype: float64